# Data Preprocessing - Instacart Dataset

Author: Kelly Li

## Table of Contents:
* [1 Introduction](#one)
* [2 Data Source](#two)
* [3 Data Exploration](#three)
    * [3.1 Handling Missing Data](#threeone) 
    * [3.2 Outlier Detection and Treatment](#threetwo)
    * [3.3 Data Anomalies](#threethree)
* [4 Data Transformation](#four)
    * [4.1 Encoding Categorical Variables](#fourone)
    * [4.2 Feature Engineering](#fourtwo)
* [5 Conclusion](#five)

---

## 1 Introduction <a class="anchor" id="one"></a>

Welcome to the Data Preprocessing phase of the Instacart Market Basket Analysis (MBA) project. In this Jupyter Notebook, we will delve into the intricacies of preparing and cleaning the Instacart dataset, laying the foundation for insightful analysis and modeling.

### Project Rationale
**Why Instacart?**

This project was inspired by a passion for data science and a genuine interest in the realm of consumer behavior and online grocery shopping. The Instacart dataset provides a rich and complex landscape of real-world data that presents a multitude of exciting analytical opportunities.

**Understanding Shopping Behavior**

The modern world of e-commerce has revolutionized the way consumers shop for everyday essentials. With the increasing reliance on online platforms for grocery shopping, understanding customer preferences, purchasing patterns, and product associations has become essential for both retailers and data scientists.

**Extracting Insights**

By undertaking this project, I aim to extract valuable insights from the dataset that can benefit both Instacart as a service provider and consumers looking for a more convenient shopping experience. Through data-driven analysis, we can uncover hidden patterns, identify trends, and develop data-driven strategies for optimizing the shopping journey.

**The Power of Data**

Data science is not merely about numbers and algorithms; it's about leveraging the power of data to solve real-world problems. Through this project, I hope to demonstrate how data preprocessing is the critical first step in transforming raw data into actionable insights. Clean and well-structured data empowers us to build accurate models, make informed decisions, and drive positive change.

### What to Expect
In this notebook, we will systematically address data quality issues, handle missing data, encode categorical variables, and prepare the dataset for subsequent stages of our analysis. Each step in the data preprocessing journey is a building block that contributes to the overarching goal of gaining a deeper understanding of customer behavior in the context of online grocery shopping.

Let's embark on this data-driven adventure and unlock the potential insights hidden within the Instacart dataset.

## 2 Data Source <a class="anchor" id="two"></a>

The [InstaCart Online Grocery Shopping Dataset](https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset) is made available for non-commercial use in 2017. This anonymized dataset comprises a substantial sample, exceeding 3 million grocery orders originating from more than 200,000 Instacart users.

The dataset presents, for each user, a range of 4 to 100 of their orders, each including the comprehensive list of products acquired within the order. Alongside the product details, it includes timestamps specifying the week and hour of the day when the order was initiated. Additionally, the dataset incorporates a relative temporal measure, serving as an indicator of the elapsed time between successive orders.

### Data Dictionary <a class="anchor" id="twoone"></a>

Here's the data description in tabular format for each dataset:

**orders.csv**

| Column Name           | Description                                                                                         |
|-----------------------|-----------------------------------------------------------------------------------------------------|
| order_id              | A unique identifier for each order.                                                                |
| user_id               | Identifies the customer associated with the order.                                                  |
| eval_set*             | Indicates the evaluation set to which the order belongs (e.g., "prior" or "train").               |
| order_number          | Represents the sequence number of the order for each user.                                          |
| order_dow             | Denotes the day of the week the order was placed.                                                   |
| order_hour_of_day     | Specifies the hour of the day when the order was placed.                                            |
| days_since_prior_order| Indicates the number of days since the last order, capped at 30 (with NAs for order_number = 1).    |

*The *eval_set* column in the "orders" dataset specifies which evaluation set each order belongs to. There are 2 possible evaluation sets:

- "prior": Orders prior to that user's most recent order (~3.2 million orders).
- "train": Training data supplied (~131,000 orders).

**order_products_train.csv**

| Column Name          | Description                                                                                            |
|----------------------|--------------------------------------------------------------------------------------------------------|
| order_id             | Links to a specific order.                                                                             |
| product_id           | Identifies the product added to the cart.                                                              |
| add_to_cart_order    | Indicates the order in which each product was added to the cart.                                       |
| reordered            | Binary indicator (1 if the product has been ordered by this user in the past, 0 otherwise).          |

**products.csv**

| Column Name     | Description                                       |
|-----------------|---------------------------------------------------|
| product_id      | Uniquely identifies each product.                  |
| product_name    | Provides the name of the product.                 |
| aisle_id        | Links to the aisle in which the product is located.|
| department_id   | Associates the product with its department.       |

**order_products_prior.csv**

| Column Name          | Description                                                                                            |
|----------------------|--------------------------------------------------------------------------------------------------------|
| order_id             | Foreign key to identify the order.                                                                      |
| product_id           | Foreign key to identify the product.                                                                    |
| add_to_cart_order    | Order in which each product was added to the cart.                                                       |
| reordered            | Binary indicator (1 if the product has been ordered by this user in the past, 0 otherwise).          |

**aisles.csv**

| Column Name | Description                            |
|-------------|----------------------------------------|
| aisle_id    | Uniquely identifies each aisle.        |
| aisle       | Provides the name of the aisle.       |

**departments.csv**

| Column Name     | Description                            |
|-----------------|----------------------------------------|
| department_id   | Uniquely identifies each department.   |
| department      | Provides the name of the department.   |


## 3 Data Exploration <a class="anchor" id="three"></a>

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

c:\Users\kelly\anaconda3\envs\bigdata_ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
orders_df = pd.read_csv("orders.csv")
orders_df.head(5)

order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

In [5]:
main_df = pd.read_csv("order_products__prior.csv")
main_df.head(5)

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

In [37]:
train_df = pd.read_csv("order_products__train.csv")
train_df

order_id  product_id  add_to_cart_order  reordered
0               1       49302                  1          1
1               1       11109                  2          1
2               1       10246                  3          0
3               1       49683                  4          0
4               1       43633                  5          1
...           ...         ...                ...        ...
1384612   3421063       14233                  3          1
1384613   3421063       35548                  4          1
1384614   3421070       35951                  1          1
1384615   3421070       16953                  2          1
1384616   3421070        4724                  3          1

[1384617 rows x 4 columns]

In [6]:
prod_df = pd.read_csv("products.csv")
prod_df.head(5)

product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13

In [7]:
aisles_df = pd.read_csv("aisles.csv")
aisles_df.head(5)

aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

In [8]:
dep_df = pd.read_csv("products.csv")
dep_df.head(5)

product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13

In [9]:
# Shape of the data
print('The shape of the following:')
print('orders dataset:', orders_df.shape)
print('order products dataset:', main_df.shape)
print('products dataset:', prod_df.shape)
print('aisles dataset:', aisles_df.shape)
print('departments dataset:', dep_df.shape)

The shape of the following:
orders dataset: (3421083, 7)
order products dataset: (32434489, 4)
products dataset: (49688, 4)
aisles dataset: (134, 2)
departments dataset: (49688, 4)


In [10]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [11]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [12]:
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [13]:
aisles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [14]:
dep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [15]:
# Convert 'id' columns to object data type
orders_df['order_id'] = orders_df['order_id'].astype(str)
orders_df['user_id'] = orders_df['user_id'].astype(str)

# Print updated data types of the columns
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 182.7+ MB


In [16]:
# Convert day of week and hour of day columns to object data type
orders_df['order_dow'] = orders_df['order_dow'].astype(str)
orders_df['order_hour_of_day'] = orders_df['order_hour_of_day'].astype(str)

# Print updated data types of the columns
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               object 
 5   order_hour_of_day       object 
 6   days_since_prior_order  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 182.7+ MB


In [17]:
# Convert 'id' columns to object data type
main_df['order_id'] = main_df['order_id'].astype(str)
main_df['product_id'] = main_df['product_id'].astype(str)

# Print updated data types of the columns
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           object
 1   product_id         object
 2   add_to_cart_order  int64 
 3   reordered          int64 
dtypes: int64(2), object(2)
memory usage: 989.8+ MB


In [18]:
# Convert 'id' columns to object data type
prod_df['product_id'] = prod_df['product_id'].astype(str)
prod_df['aisle_id'] = prod_df['aisle_id'].astype(str)
prod_df['department_id'] = prod_df['department_id'].astype(str)

# Print updated data types of the columns
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  object
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  object
 3   department_id  49688 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [19]:
# Convert 'id' columns to object data type
aisles_df['aisle_id'] = aisles_df['aisle_id'].astype(str)

# Print updated data types of the columns
aisles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    object
 1   aisle     134 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


In [20]:
# Convert 'id' columns to object data type
dep_df['product_id'] = dep_df['product_id'].astype(str)
dep_df['aisle_id'] = dep_df['aisle_id'].astype(str)
dep_df['department_id'] = dep_df['department_id'].astype(str)

# Print updated data types of the columns
dep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  object
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  object
 3   department_id  49688 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [21]:
# Checking for missing values
orders_df.isna().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [22]:
orders_df.head(5)

order_id user_id eval_set  order_number order_dow order_hour_of_day  \
0  2539329       1    prior             1         2                 8   
1  2398795       1    prior             2         3                 7   
2   473747       1    prior             3         3                12   
3  2254736       1    prior             4         4                 7   
4   431534       1    prior             5         4                15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0

In [28]:
nulls = orders_df[pd.isna(orders_df['days_since_prior_order'])]
nulls.head(5)

order_id user_id eval_set  order_number order_dow order_hour_of_day  \
0   2539329       1    prior             1         2                 8   
11  2168274       2    prior             1         2                11   
26  1374495       3    prior             1         1                14   
39  3343014       4    prior             1         6                11   
45  2717275       5    prior             1         3                12   

    days_since_prior_order  
0                      NaN  
11                     NaN  
26                     NaN  
39                     NaN  
45                     NaN

In [29]:
filtered = main_df[main_df['order_id'] == '2539329']
filtered

order_id product_id  add_to_cart_order  reordered
24076664  2539329        196                  1          0
24076665  2539329      14084                  2          0
24076666  2539329      12427                  3          0
24076667  2539329      26088                  4          0
24076668  2539329      26405                  5          0

NaN values in the 'days_since_prior_order' column indicate the absence of a prior order for a particular customer within the dataset.

It signifies that there are specific conditions regarding the order's placement:
- **First Order**: The most common scenario where 'days_since_prior_order' is NaN is when a customer places their very first order. Since there is no prior order for this customer in the dataset, there are no days to measure since the last order.
- **Single-Time Shoppers**: Another scenario is when a customer places a single order and does not make any subsequent orders within the dataset's time frame. In this case, there are no "prior" orders to calculate days between, resulting in a NaN value.

In [24]:
# Checking for missing values
main_df.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [31]:
# Duplicated rows
print("Duplicated rows in:")
print("orders dataset:", orders_df.duplicated().sum())
print("order products dataset:", main_df.duplicated().sum())
print("products dataset:", prod_df.duplicated().sum())
print("aisles dataset:", aisles_df.duplicated().sum())
print("departments dataset:", dep_df.duplicated().sum())

Duplicated rows in:
orders dataset: 0
order products dataset: 0
products dataset: 0
aisles dataset: 0
departments dataset: 0


In [32]:
# Summary statistics of orders dataset
orders_df.describe()

order_number  days_since_prior_order
count  3.421083e+06            3.214874e+06
mean   1.715486e+01            1.111484e+01
std    1.773316e+01            9.206737e+00
min    1.000000e+00            0.000000e+00
25%    5.000000e+00            4.000000e+00
50%    1.100000e+01            7.000000e+00
75%    2.300000e+01            1.500000e+01
max    1.000000e+02            3.000000e+01

In [33]:
# Summary statistics of order products dataset
main_df.describe()

add_to_cart_order     reordered
count       3.243449e+07  3.243449e+07
mean        8.351076e+00  5.896975e-01
std         7.126671e+00  4.918886e-01
min         1.000000e+00  0.000000e+00
25%         3.000000e+00  0.000000e+00
50%         6.000000e+00  1.000000e+00
75%         1.100000e+01  1.000000e+00
max         1.450000e+02  1.000000e+00

In [44]:
count = orders_df.groupby('eval_set')['user_id'].count()
count

eval_set
prior    3214874
test       75000
train     131209
Name: user_id, dtype: int64

In [46]:
# Sort the Series in descending order
count_sorted = count.sort_values(ascending=False)

# Create the barplot
count_sorted.plot(kind='bar', rot=0)
plt.xlabel('Evaluation Set')
plt.ylabel('User Counts')
plt.title('User Counts by Evaluation Set')
plt.show()

In [36]:
unique_count = orders_df.groupby('eval_set')['user_id'].nunique()
unique_count

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [43]:
# Sort the Series in descending order
unique_count_sorted = unique_count.sort_values(ascending=False)

# Create the barplot
unique_count_sorted.plot(kind='bar', rot=0)
plt.xlabel('Evaluation Set')
plt.ylabel('Unique User Counts')
plt.title('Unique User Counts by Evaluation Set')
plt.show()

In the dataset, there is a total of 206,209 unique customers. Among these, 131,209 customers' most recent purchases have been designated as the 'train' set. The objective is to use this training data to develop predictive models. Subsequently, the remaining 75,000 customers are the focus of prediction, where we aim to forecast their future purchasing behavior based on their prior shopping patterns.